In [1]:
import torch

# Humandoid MARL
from Humanoid_MARL import envs
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.visual import save_video, save_rgb_image
from Humanoid_MARL.utils.torch_utils import save_models, load_models
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs

In [2]:
config = {
        'num_timesteps': 100_000_000,
        'eval_frequency': 10,
        'episode_length': 1000,
        'unroll_length': 10,
        'num_minibatches': 32,
        'num_update_epochs': 8,
        'discounting': 0.97,
        'learning_rate': 3e-4,
        'entropy_cost': 1e-3,
        'num_envs': 2048,
        'batch_size': 512,
        'env_name': "humanoids",
        'render' : True,
        'device' : 'cuda',
        'model_path' : "../models/20240205_193432_ppo.pt",
        'video_length' : 300,
    }
env = envs.create(
        config['env_name'], batch_size=None, episode_length=None, backend="generalized", auto_reset=False,
    )
env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['device'], get_jax_state=True)

# env warmup
obs = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['device'])
env.step(action)
agents = load_models(config['model_path'], Agent, device=config['device'])
jax_states = []
num_steps = 1000

eval_reward = 0.0
for i in range(num_steps):
    print(f"{i} / {num_steps}")
    logits, action = get_agent_actions(agents, obs, env.obs_dims)
    jax_state, observation, reward, done, info = env.step(Agent.dist_postprocess(action))
    jax_states.append(jax_state)
    print(f"{i} | {info}")
    eval_reward += reward
print(f"Total Reward | {eval_reward}")


Models loaded from ../models/20240205_193432_ppo.pt
0 / 1000
0 | {'distance_from_origin': tensor([0.9494, 1.3813], device='cuda:0'), 'forward_reward': tensor([ 0.0001, -0.0162], device='cuda:0'), 'reward_alive': tensor([5., 5.], device='cuda:0'), 'reward_linvel': tensor([ 0.0001, -0.0162], device='cuda:0'), 'reward_quadctrl': tensor([-1.6854, -1.6939], device='cuda:0'), 'x_position': tensor([0.0197, 1.0185], device='cuda:0'), 'x_velocity': tensor([ 9.8969e-05, -1.2978e-02], device='cuda:0'), 'y_position': tensor([0.0003, 0.0026], device='cuda:0'), 'y_velocity': tensor([ 0.0099, -0.0012], device='cuda:0')}
1 / 1000
1 | {'distance_from_origin': tensor([0.9438, 1.3774], device='cuda:0'), 'forward_reward': tensor([-0.0014, -0.0162], device='cuda:0'), 'reward_alive': tensor([5., 5.], device='cuda:0'), 'reward_linvel': tensor([-0.0014, -0.0162], device='cuda:0'), 'reward_quadctrl': tensor([-1.6847, -1.6893], device='cuda:0'), 'x_position': tensor([0.0197, 1.0183], device='cuda:0'), 'x_veloci

In [6]:
env = envs.get_environment(env_name=config['env_name'],
                           backend='generalized')

In [7]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 